# Contract analytics for Cheese Grotto

In [1]:
import numpy as np
import pandas as pd
import requests
import time
import matplotlib.pyplot as plt
from pandasql import sqldf #for manipulating DataFrames with SQL style queries
#Write file to machine to work with offline
import json


# Instagram graph api

In [60]:
#generated each session by the graph api explorer
user_access_token = ''

In [62]:
#Get the user's pages
#For better security these must all be derived after a new user access token is created by the person logged 
#into the fb business account
user_pages = f'https://graph.facebook.com/v4.0/me/accounts?access_token={user_access_token}'
response = requests.get(user_pages)
page_id = json.loads(response.text)['data'][0]['id']

#Get the page's instagram business account id
insta_biz_account = f'https://graph.facebook.com/v4.0/{page_id}?fields=instagram_business_account&access_token={user_access_token}'
response = requests.get(insta_biz_account)
insta_id = json.loads(response.text)['instagram_business_account']['id']

In [63]:
insta_id

'17841401233892528'

In [85]:
#Get the Instagram Business Account's Media Objects
media_objects = f'https://graph.facebook.com/v4.0/{insta_id}/media?access_token={user_access_token}&limit=2000'
response = requests.get(media_objects)
media_object_id = json.loads(response.text)['data'][0]['id']

In [86]:
len(json.loads(response.text)['data'])

1317

In [88]:
json.loads(response.text)['data'][1316]

{'id': '17841922594041130'}

In [89]:
#Get oldest media object to test
oldest = json.loads(response.text)['data'][1316]['id']

In [67]:
#First media object
media_object_id

'18032803363216401'

In [75]:
#SEe for more fields: https://developers.facebook.com/docs/instagram-api/reference/media/

requests.get(f'https://graph.facebook.com/v4.0/{media_object_id}?fields=comments,like_count,timestamp&access_token={user_access_token}').text
#/?fields={fields}

'{"comments":{"data":[{"timestamp":"2019-10-08T20:09:43+0000","text":"#avantrevivalists #ruralistmodern #howiseeluxury #howisummer #winecellar #rawmilkcheese #cheeseforlife #wineandcheese #creativityfound #whiskeyporn #cheeseandwine #cheeseknife #thatcheeseplate #doitfortheprocess #mycommontable #abmlifeissweet #buzzfeedtasty #mybeautifulmess #tastemade #foundforaged #bedeeplyrooted #thatcolorproject #howyouglow #theblogissue #makeitdelicious #abmathome #makeyousmilestyle #bandofun #mydomaine #whiskylover","id":"17853780745589206"}]},"like_count":8,"timestamp":"2019-10-08T20:09:25+0000","id":"18032803363216401"}'

In [90]:
#Test oldest post
requests.get(f'https://graph.facebook.com/v4.0/{oldest}?fields=comments,like_count,timestamp&access_token={user_access_token}').text


'{"comments":{"data":[{"timestamp":"2014-11-11T00:20:51+0000","text":"\\u0040foodandwine","id":"17850626032041130"}]},"like_count":8,"timestamp":"2014-11-10T22:47:02+0000","id":"17841922594041130"}'